# Simple Dicom Visualizer

This is a little visualizer to easily and quickly look at the training data. This should come in handy during this competition, when a quick look at the images of a specific patient is needed. Feel free to use and change this to your heart's content :)

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact
import pydicom as dcm
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.express as px

Path.ls = lambda p: list(p.iterdir())

In [ ]:
p_data = Path("/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification")
p_train_labels = p_data/"train_labels.csv"
p_train = p_data/"train"

In [ ]:
label_df = pd.read_csv(p_train_labels)

In [ ]:
type_idx = 0
patient_idx = 0
patient = None

modalities = ["T2w", "T1wCE", "T1w", "FLAIR"]
train_patients = label_df["BraTS21ID"].values

dcm_images = None

@interact
def selections(Modality=modalities, BraTS21ID=train_patients):
    global type_idx, patient_idx, patient, dcm_images
    
    patient_idx = np.where(label_df["BraTS21ID"].values == BraTS21ID)[0]
    
    assert len(patient_idx) < 2, f"{BraTS21ID}\n{patient_idx}"
    patient_idx = patient_idx[0]
    
    patient = sorted(p_train.ls(), key=lambda p: int(p.stem))[patient_idx]
    m2i = {p.stem: i for i, p in enumerate(patient.ls())}
    type_idx = m2i[Modality]
    
    dcm_images = (patient.ls()[type_idx]).ls()
    dcm_images = sorted(dcm_images, key = lambda p: int(p.stem.split("-")[-1]))
    dcm_images = [dcm.dcmread(p).pixel_array for p in dcm_images]

In [ ]:
# Rerun this cell everytime you change the values above

patient_label = label_df[label_df["BraTS21ID"] == int(patient.name)]["MGMT_value"].values[0]

def viewer(image_idx):
    print(f"BraTS21ID: {patient.name}\tMGMT_value: {patient_label}\tModality: {patient.ls()[type_idx].stem}")
    fig = px.imshow(dcm_images[image_idx], binary_string=True)
    fig.show()

interact(viewer, 
         image_idx=widgets.IntSlider(min=0, max=len(dcm_images)-1, step=1, value=0));